In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import csv

In [ ]:
transaction = pd.read_csv("transactions.csv", delimiter="|")
items = pd.read_csv("items.csv", delimiter="|")

In [ ]:
transaction.head()

In [ ]:
print(transaction.shape)
print(items.shape)

In [ ]:
data = pd.merge(items, transaction, on=["itemID"], how="outer")

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
# choosing Target variable
#print(inner_merged_total['basket'].value_counts())
#print(inner_merged_total['click'].value_counts())
print(data['order'].value_counts())

In [ ]:
##data preprocessing 
##generate target variable interest
data["Interest"] = data["click"] + data["basket"]+ data["order"]


## aggregrate (idea is to generate the frequencies of item id, and multiply it by interest, getting the total interest of the items)

In [ ]:
data.head()
#print(inner_merged_total['Interest'].value_counts())

## Group by item ID, and then count the item ID, and then interest * count of itemID = total interest column

In [ ]:
##generate frequency
test = data.value_counts(['itemID']).reset_index(name='Frequency')
print(test.head())

In [ ]:
#Merging frequency 
book_data = pd.merge(data, test, on=["itemID"])
book_data.head()

In [ ]:
#remove dupicate rows 
print(book_data.shape)
# book_data.drop_duplicates(subset ="itemID", keep='first' ,inplace=True)
# print(book_data.shape)

In [ ]:
book_data["itemID"].value_counts()

In [ ]:
book_data.head()

In [ ]:
book_rating= book_data.copy()
book_rating.drop(['sessionID', 'click','author','publisher'], axis=1)

In [ ]:
##data cleaning
values = {'basket': 0, 'order': 0}
book_rating.fillna(value=values)

In [ ]:
import numpy as np
book_rating['log_frequency'] = np.log(book_data['Frequency'])

In [ ]:
print(book_rating['log_frequency'].value_counts())

In [ ]:
book_rating['log_frequency'].describe()

In [ ]:
bin_labels_5 = ['E','D','C', 'B', 'A']
book_rating['Grade'] = pd.qcut(book_rating['log_frequency'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5, duplicates='drop')
book_rating


In [ ]:
book_rating = book_rating.drop(['sessionID', 'click','author','publisher','Frequency'], axis=1)

In [ ]:
book_rating.drop_duplicates(subset ="title", keep='first' ,inplace=True)

In [ ]:
book_rating = book_rating.reset_index()

In [ ]:
book_rating

In [ ]:
#creating combined target feature
def parseFeature(book_rating):
    data = str(book_rating["main topic"]) + ','
    data += str(book_rating["subtopics"]) + ','
    data += str(book_rating['relevance_score'])
    x = data.replace('[','')
    y = x.replace(']','')
    return y
    
    

In [ ]:
#calculate relevance score

def relevance(book_rating):
    if (book_rating['order'] != 0):
        return "A"
    elif (book_rating['order'] == 0 and  book_rating['basket'] != 0):
        return "B"
    else:
        return "C"
    

book_rating['relevance_score'] = book_rating.apply(relevance, axis = 1)


In [ ]:
book_rating['parsedFeatureStr'] = book_rating.apply(parseFeature, axis = 1)

In [ ]:
book_rating.head(40)

In [ ]:
book_rating.drop_duplicates(subset ="itemID", keep='first' ,inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

c_vectorizer = CountVectorizer(min_df=1,
                            strip_accents='unicode', analyzer='word',token_pattern=r',',ngram_range=(1,3),stop_words='english')

vectorizer = TfidfVectorizer(min_df=3, max_features=None,
                            strip_accents='unicode', analyzer='word',token_pattern=r',',ngram_range=(1,3),stop_words='english')

h_vectorizer = HashingVectorizer(n_features=2**1,
                            strip_accents='unicode', analyzer='word',token_pattern=r',',ngram_range=(1,3),stop_words='english')

In [ ]:
parsedFeatureMatrix = vectorizer.fit_transform(book_rating['parsedFeatureStr'])

In [ ]:
parsedFeatureMatrix

In [ ]:
parsedFeatureMatrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel 
from sklearn.metrics.pairwise import cosine_similarity 

sig = sigmoid_kernel(parsedFeatureMatrix, parsedFeatureMatrix)
# cos = cosine_similarity(parsedFeatureMatrix, parsedFeatureMatrix)

In [ ]:
sig[0]

In [ ]:
indices = pd.Series(book_rating.index, index = book_rating['title'])

In [ ]:
indices_df = pd.Series(book_rating.index, index = book_rating['title']).to_frame()

In [ ]:
indices_itemID_mapping = pd.merge(indices_df, book_rating, on=["title"])

In [ ]:
indices_df

In [ ]:
indices_itemID_mapping = indices_itemID_mapping.drop(['index'], axis=1)

In [ ]:
indices_itemID_mapping

In [ ]:
def recommender(title, sig=sig):
    index = indices[title]
    print(index)
    
    # get similarity scores
    sig_scores = list(enumerate(sig[index]))
    #sort by scores
    sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:6]
    
    #relate to book
    book_indices = [i[0] for i in sig_scores]
    
    return book_rating['itemID'].iloc[book_indices]
    
    
    

In [ ]:
recommender_df = recommender('Mein großes Schablonen-Buch - Wilde Tiere').to_frame()

In [ ]:
recommender_df

In [ ]:
t = pd.merge(recommender_df, book_rating, on=["itemID"])

In [ ]:
t

## Populate evaluation DF


In [ ]:
## Use Evaluation dataset to recommend 5 itemID

evaluation_df = pd.read_csv("evaluation.csv", delimiter="|")

evaluation_df

In [ ]:
title_df = indices_itemID_mapping.drop(['basket','order','main topic','subtopics','log_frequency','Grade', 'relevance_score', 0, 'parsedFeatureStr'], axis=1)
title_df

In [ ]:
submission_df = pd.merge(evaluation_df, items, on=["itemID"]) 
submission_df = submission_df.drop(['author','publisher','main topic','subtopics'], axis=1)
submission_df

In [ ]:
def getSigScoreArray(num, title, sig=sig):
    index = indices[title]
    print(index)
    
    # get similarity scores
    sig_scores = list(enumerate(sig[index]))
    #sort by scores
    sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:num+1]
    
    test_sig_score_arr = []
    test_sig_score_arr = sig_scores
    
    sig_score_array = []
    for score in test_sig_score_arr:
        sig_score_array.append(score[1])
    return sig_score_array

In [ ]:
sig_score_array = getSigScoreArray(5, "Princess Poppy: The Big Mix Up")

In [ ]:
##Create new column where order != 0 but basket and/or click is 0

def identify(book_data):
    if (book_data['order'] != 0 and (book_data['basket']  | book_data['click'] == 0)):
        return 'Found'
    else:
        return 'Not found'

book_data['test'] = book_data.apply(identify, axis = 1)

book_data.head(10)

In [ ]:
#Print how many counts for order column

print(book_data['order'].value_counts())


In [ ]:
## Print how many counts for test column

print(book_data['test'].value_counts())


In [ ]:
## Print column where test == found

test_column = book_data.loc[book_data['test'] == 'Found']

test_column.head(10)


In [ ]:
##clone dataframe
sea_born_df = book_data.copy(deep=True)

In [ ]:
sea_born_df.shape

In [ ]:
## dropping unnessesary columns for pairplot
sea_born_df = sea_born_df.filter(items = ['click', 'basket', 'order'])


In [ ]:
## Make pair plot using seaborn
import seaborn as sns


# sns.set(style="ticks", color_codes=True)
sns.pairplot(sea_born_df)


import matplotlib.pyplot as plt
plt.show()



In [ ]:
#Create histogram -> edited

x = book_data['Total_Interest'].value_counts()

plt.hist(x, bins = 60)
plt.show()

In [ ]:
## remove interest, total interest, click, basket, order from dataframe

latest_data = book_data.drop(['Interest', 'Total_Interest', 'click', 'basket', 'order'], axis = 1)
latest_data


In [ ]:
## EDA main topic vs subtopic vs title and use only frequency


In [ ]:
#test on printing rows with one main topic

latest_data.loc[latest_data['main topic'] == 'FM'].head(5)


In [ ]:
# Show title, main topic & subtopics with frequency value in mean
print(latest_data.groupby('title')['Frequency'].mean().sort_values(ascending=False).head(), '\n')
print(latest_data.groupby('main topic')['Frequency'].mean().sort_values(ascending=False).head(), '\n')
print(latest_data.groupby('subtopics')['Frequency'].mean().sort_values(ascending=False).head())


In [ ]:
# Show title, main topic & subtopics with frequency value in counts
print(latest_data.groupby('title')['Frequency'].count().sort_values(ascending=False).head(), '\n')
print(latest_data.groupby('main topic')['Frequency'].count().sort_values(ascending=False).head(), '\n')
print(latest_data.groupby('subtopics')['Frequency'].count().sort_values(ascending=False).head())

In [ ]:
#Mean value for title
title = pd.DataFrame(latest_data.groupby('title')['Frequency'].mean().sort_values(ascending=False))
#Count value for title
title['Counts'] = pd.DataFrame(latest_data.groupby('title')['Frequency'].count())
title


In [ ]:
print(title['Counts'].value_counts(), '\n')    #Counts
print(title['Frequency'].value_counts(), '\n') #Mean 

In [ ]:
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [ ]:
plt.figure(figsize=(10,4))
title['Counts'].hist(bins=30)

plt.figure(figsize=(10,4))
title['Frequency'].hist(bins=80)

In [ ]:
sns.jointplot(x='Counts', y='Frequency', data=title,  alpha=0.6)

In [ ]:
#Mean value for main topic
main = pd.DataFrame(latest_data.groupby('main topic')['Frequency'].mean().sort_values(ascending=False))
#Count value for main topic
main['Counts'] = pd.DataFrame(latest_data.groupby('main topic')['Frequency'].count())
main

In [ ]:
#Show latest data
print(latest_data['main topic'].value_counts(), '\n')    #Counts

#Show the counts for duplicate main topic
#You may ignore this ...... Not sure if this is good or not
print('Number of duplicated rows:', latest_data.duplicated(['main topic'], keep='first').value_counts(), '\n')

#Show where total duplicated main topic more than 5
duplicate = latest_data['main topic'].value_counts()
print(duplicate.loc[duplicate > 5])

#Show where total duplicated main topic less than or equal to 5
duplicate.loc[duplicate <= 5]


In [ ]:
print(main['Counts'].value_counts(), '\n')    #Counts
print(main['Frequency'].value_counts(), '\n') #Mean 

In [ ]:
plt.figure(figsize=(10,4))
main['Counts'].hist(bins=30)

plt.figure(figsize=(10,4))
main['Frequency'].hist(bins=70)

In [ ]:
sns.jointplot(x='Counts', y='Frequency', data=main,  alpha=0.6)

In [ ]:
#Mean value for subtopic
sub = pd.DataFrame(latest_data.groupby('subtopics')['Frequency'].mean().sort_values(ascending=False))
#Count value for subtopic
sub['Counts'] = pd.DataFrame(latest_data.groupby('subtopics')['Frequency'].count())
sub

In [ ]:
print(sub['Counts'].value_counts(), '\n')    #Counts
print(sub['Frequency'].value_counts(), '\n') #Mean 

In [ ]:
plt.figure(figsize=(10,4))
sub['Counts'].hist(bins=10)

plt.figure(figsize=(10,4))
sub['Frequency'].hist(bins=70)

In [ ]:
sns.jointplot(x='Counts', y='Frequency', data=sub,  alpha=0.6)

## Extra EDA

In [ ]:
transaction = pd.read_csv("transactions.csv", delimiter="|")
items = pd.read_csv("items.csv", delimiter="|")

In [ ]:
data = pd.merge(transaction, items, on=["itemID"])

In [ ]:
data["Interest"] = data["click"] + data["basket"]+ data["order"]

In [ ]:
##generate frequency
test = data.value_counts(['itemID']).reset_index(name='Frequency')
print(test.head())

In [ ]:
#Merging frequency 
book_data = pd.merge(data, test, on=["itemID"])
book_data.head()

In [ ]:
## removing unnnessesarry column
del book_data["click"]
del book_data["basket"]
del book_data["order"]

In [ ]:
book_data.head()

In [ ]:
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [ ]:
book_data.groupby('title')['Interest'].mean().sort_values(ascending=False).head()

In [ ]:
book_data.groupby('title')['Interest'].count().sort_values(ascending=False).head()

In [ ]:
# interests = pd.DataFrame(book_data.groupby('title')['Interest'].mean())
# interests = pd.DataFrame(book_data.groupby('title')['click'].mean())
# #interests = pd.DataFrame(book_data.groupby('title')['basket'].mean())
interests = pd.DataFrame(book_data.groupby('title')['Interest'].mean())
interests.head()

In [ ]:
# interests['number of interests'] = pd.DataFrame(book_data.groupby('title')['Interest'].count())
# interests['number of click'] = pd.DataFrame(book_data.groupby('title')['click'].count())
# interests['number of basket'] = pd.DataFrame(book_data.groupby('title')['basket'].count())
interests['number of order'] = pd.DataFrame(book_data.groupby('title')['Interest'].count())
interests.head()

In [ ]:
plt.figure(figsize=(10,4))
interests['number of interests'].hist(bins=70)

In [ ]:
plt.figure(figsize=(10,4))
interests['Interest'].hist(bins=70)

In [ ]:
sns.jointplot(x='Interest', y='number of interests', data=interests,  alpha=0.5)

In [ ]:
sns.jointplot(x='order', y='number of basket', data=interests,  alpha=0.5)

In [ ]:
sns.jointplot(x='basket', y='number of basket', data=interests,  alpha=0.5)

In [ ]:
sns.jointplot(x='click', y='number of click', data=interests,  alpha=0.5)

## Pivot Table

In [ ]:
book_pivot = book_data_test.pivot_table(index='sessionID', columns='title', values='Interest')
book_pivot.head()

In [ ]:
book_data_test = book_data.head(100000)

In [ ]:
interests.sort_values('number of interests', ascending=False).head(10)